In [1]:
import tensorflow as tf

## 0.前言
维度变换的原因：不同shape的矩阵相加需要统一shape

## 1.改变视图

从不同的角度观察数据，产生不同的视图。如shape为[2,4,4,3]的张量A：
1. 逻辑上，可以理解为2张图片，每张图片4行4列，每个位置有RGB3个通道的数据；
2. 也可以理解为2个样本，每个样本的特征为长度48的向量

内存没有维度这一概念，只以平铺方式写入内存，因此多维度的层级关系需要认为管理

In [2]:
x=tf.range(96) # 模拟生成一个向量数据
x1=tf.reshape(x,[2,4,4,3]) # 改变视图，获得4D张量，存储未改变
print(x)
print(x1)
print(id(x))
print(id(x1))
print(x.ndim)
print(x1.ndim)
print(x.shape)
print(x1.shape)

tf.Tensor(
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95], shape=(96,), dtype=int32)
tf.Tensor(
[[[[ 0  1  2]
   [ 3  4  5]
   [ 6  7  8]
   [ 9 10 11]]

  [[12 13 14]
   [15 16 17]
   [18 19 20]
   [21 22 23]]

  [[24 25 26]
   [27 28 29]
   [30 31 32]
   [33 34 35]]

  [[36 37 38]
   [39 40 41]
   [42 43 44]
   [45 46 47]]]


 [[[48 49 50]
   [51 52 53]
   [54 55 56]
   [57 58 59]]

  [[60 61 62]
   [63 64 65]
   [66 67 68]
   [69 70 71]]

  [[72 73 74]
   [75 76 77]
   [78 79 80]
   [81 82 83]]

  [[84 85 86]
   [87 88 89]
   [90 91 92]
   [93 94 95]]]], shape=(2, 4, 4, 3), dtype=int32)
140654373151744
140654373151264
1
4
(96,)
(2, 4, 4, 3)


`tf.reshape(x,[2,-1])`中的-1表示对应维度自动推导。

In [3]:
print(tf.reshape(x,[2,-1]))

tf.Tensor(
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
  24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
  72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]], shape=(2, 48), dtype=int32)


## 2.增删维度

增加维度并非修改数据，只是加一个长度为1的维度，改变数据的理解方式
 * 通过`tf.expand_dims(x,axis)`可以在指定的axis轴前插入一个新维度，axis可为负数

In [4]:
x=tf.random.uniform([28,28],maxval=10,dtype=tf.int32)
x1=tf.expand_dims(x,axis=2)
print(x1.shape)
x2= tf.expand_dims(x1,axis=0)
print(x2.shape)

(28, 28, 1)
(1, 28, 28, 1)


删除维度是增加维度的逆操作，只能输出长度为1的维度，不改变张量的存储。
* 通过`tf.squeeze(x,axis)`删除指定轴的维度，如果axis不提供，默认删除所有长度为1的维度

In [5]:
x3=tf.squeeze(x2,axis=0)
print(x3.shape)
x4=tf.squeeze(x3,axis=2)
print(x4.shape)

(28, 28, 1)
(28, 28)


## 3.交换维度

使用`tf.transpose(x,perm)`完成，perm表示新的维度顺序List
注：维度交换后，张量的存储顺序也改变，比改变视图的计算代价更高


In [6]:
x=tf.random.normal([2,32,32,3])
x1=tf.transpose(x,perm=[0,3,1,2])
print(x1.shape)

(2, 3, 32, 32)


## 4.复制数据

典型的操作是先增加长度为1的维度，然后在该维度进行复制若干份数据
* `tf.tile(x,multiples)`，multiples表示每个维度上复制的倍数，1表示不复制，2表示原来长度的2倍。
* `tf.tile`会创建一个新的张量来保存复制后的张量，由于复制操作涉及大量数据的读写IO运算，计算代价较高。

In [7]:
b=tf.constant([3,7])
print(b)
b1=tf.expand_dims(b,axis=0)
print(b1)
b2=tf.tile(b1,multiples=[2,1])
print(b2)

tf.Tensor([3 7], shape=(2,), dtype=int32)
tf.Tensor([[3 7]], shape=(1, 2), dtype=int32)
tf.Tensor(
[[3 7]
 [3 7]], shape=(2, 2), dtype=int32)


## 5.Broadcasting

广播机制，相对于`tf.tile`是一种轻量级复制手段，在逻辑上扩展张量数据的形状，但只会在需要时才执行实际存储复制操作。
* Broadcasting 机制的核心思想是普适性,即同一份数据能普遍适合于其他位置。在验证普适性之前,需要先将张量 shape 靠右对齐,然后进行普适性判断:对于长度为 1 的维度,默认这个数据普遍适合于当前维度的其他位置;对于不存在的维度,则在增加新维度后默认当前数据也是普适于新维度的,从而可以扩展为更多维度数、任意长度的张量形状。
* 对于用户而言，Broadcasting和tf.tile复制的最终效果一样，操作对用户透明，但前者节省资源
* Broadcasting流程：

![](https://github.com/zfhxi/Learn_tensorflow/blob/master/TensorFlowDL/ch04-TensorFlow%E5%9F%BA%E7%A1%80/img/03.png?raw=true)

考虑$Y=X@W+b$，$X@W$的shape为\[2,3\]，b的shape为\[3\]，使用广播机制复制数据如下：

In [8]:
X=tf.random.normal([2,4])
W=tf.random.normal([4,3])
b=tf.random.normal([3])
Y=X@W+b
print(Y.shape)

(2, 3)


这里进行`+`操作时，自动调用了`tf.broadcast_to(b,[2,3])`的Broadcasting机制

也可以主动使用`tf.broadcast_to(x,new_shape)`显式地进行自动扩展

In [9]:
A=tf.random.normal([32,1])
print(A.shape)
A1=tf.broadcast_to(A,[2,32,32,3])
print(A1.shape)

(32, 1)
(2, 32, 32, 3)


普适性失败案例：

![](https://github.com/zfhxi/Learn_tensorflow/blob/master/TensorFlowDL/ch04-TensorFlow%E5%9F%BA%E7%A1%80/img/03.png?raw=true)

In [10]:
A=tf.random.normal([32,2])
try:
    A1=tf.broadcast_to(A,[2,32,32,4])
except Exception as e:
    print(e)

Incompatible shapes: [32,2] vs. [2,32,32,4] [Op:BroadcastTo]


In [ ]:
import os
pid=os.getpid()
!kill -9 $pid

